The code in this notebook was mainly modified from https://github.com/pytorch/examples/blob/master/mnist/main.py

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [20]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [21]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
log_interval = 10

In [10]:
# set CUDA computing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
# load data
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)

In [13]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [24]:
model = Net().to(device)
# use multi-GPUs
model = nn.DataParallel(model)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [25]:
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.337183
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.242791
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.173599
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.028429
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.842776
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.373316
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.980961
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.826316
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.703372
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.644808
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.453746
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.636141
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.305338
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.356823
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.243398
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.378249
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.347061
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.290357
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.312700
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.331665
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.087587
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.098563
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.131705
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.063931
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.153358
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.087489
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.120399
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.067058
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.039965
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.118949
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.026903
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.104108
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.088250
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.033212
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.049616
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.155154
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.172986
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.073228
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.120666
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.038895


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.062012
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.016632
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.028205
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.021876
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.022950
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.119139
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.038193
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.065042
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.008849
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.016898
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.194708
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.013281
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.153830
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.099648
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.122737
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.019310
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.008010
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.115468
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.029858
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.018053


Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.008126
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.024448
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.100519
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.086399
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.014651
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.006291
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.034129
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.001978
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.052109
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.010782
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.163501
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.013100
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.035119
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.007179
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.016303
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.071479
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.008208
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.057870
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.037632
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.008144


Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.009315
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.021761
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.113920

Test set: Average loss: 0.0336, Accuracy: 9891/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.025756
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.029529
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.003816
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.004390
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.003942
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.016938
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.015553
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.025387
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.060345
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.001384
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.001234
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.047180
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.009956
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.025821
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.002846
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.018662
Train Epoch: 8 [10

Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.018874
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.004766
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.014625
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.002624
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.021594
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.006114
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.002208
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.013134
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.062219
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.043317
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.002442
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.008904
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.019081
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.004749
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.035424
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.009140
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.005837
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.007161
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.011904
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.009978
